In [1]:
import os
import numpy as np
os.environ['PYSPARK_SUBMIT_ARGS'] = '--conf spark.ui.port=4041 --packages org.apache.kafka:kafka_2.11:0.9.0.0,org.apache.kafka:kafka-clients:0.9.0.0  pyspark-shell'

In [2]:
from pyspark import SparkContext
sc = SparkContext("local[1]", "KafkaSendStream") 
from kafka import KafkaProducer
import time

In [ ]:
import json
from satori.rtm.client import make_client, SubscriptionMode
from math import log

# Make a proxy class

endpoint = "wss://open-data.api.satori.com"
appkey = "dC6c33Fbb5ECdAC1Ef2aB77dcBfBB0B0"
channel = "cryptocurrency-market-data"

with make_client(endpoint=endpoint, appkey=appkey) as client:
    print('Connected to Satori RTM!')
    messagebox=[]

    class SubscriptionObserver(object):
        def on_subscription_data(self, data):
            for message in data['messages']:
                messagebox.append(message)

    subscription_observer = SubscriptionObserver()
    client.subscribe(
        channel,
        SubscriptionMode.SIMPLE,
        subscription_observer,
        {'filter':'select * from `cryptocurrency-market-data` where exchange = "Bitstamp" and basecurrency = "USD" and cryptocurrency="BTC"'})
        #  exchange != "ShapheShift" OU type != "ratequote" si on laisse ShapeShift

    producer = KafkaProducer(bootstrap_servers='localhost:9092')
    i=0
    numOfColumnsX=3   # number of columns of X counting t_0 column with only ones
    numOfRendements=10
    timeBetweenPrices=3
    while not messagebox: # waiting for a first price to arrive
        time.sleep(1)
    firstMessage=messagebox.pop()
    firstMessage=json.dumps(firstMessage, ensure_ascii=False)
    firstMessage=json.loads(firstMessage)
    lastPrice=float(firstMessage['price']) # get that first price
    lastTime=int(firstMessage['timestamp'])
    rendements=np.empty(0)
    proxies=np.empty(0)
    interpolatedTimes=[]
    interpolatedPrices=[]
    while True:
        proxies=proxies[1:]
        while proxies.size<numOfColumnsX: # Pas -1 car le dernier proxy sera utilisé comme y            
            if messagebox and not interpolatedTimes:
                newMessage=messagebox.pop()
                newMessage=json.dumps(newMessage, ensure_ascii=False)
                newMessage=json.loads(newMessage)
                prices=[lastPrice, float(newMessage['price'])]
                times=[lastTime, int(newMessage['timestamp'])]
                interpolatedTimes=list(range(lastTime, int(newMessage['timestamp']), timeBetweenPrices))
                interpolatedPrices=list(np.interp(interpolatedTimes, times, prices))
            while interpolatedTimes and proxies.size<numOfColumnsX:
                newPrice=interpolatedPrices.pop(0)
                rendement=np.log(newPrice/lastPrice)
                lastPrice=newPrice
                lastTime=interpolatedTimes.pop(0)
                rendements=np.append(rendements, rendement)
                if rendements.size==numOfRendements:
                    proxies=np.append(proxies,np.sum(np.square(rendements))/numOfRendements)
                    rendements=rendements[1:]
            time.sleep(1)
    
        y=proxies[-1]*1000000
        message=np.array2string(np.append([i,y], np.insert(proxies[:numOfColumnsX-1]*1000000,0,1)),separator=",") # Ajout de la colonne t_0 aux proxies 
        producer.send('volatility', message)
        i=i+1
        print(message)

Connected to Satori RTM!
[ 0.        , 0.0345133 , 1.        , 0.03260149, 0.03763058]
[ 1.        , 0.03933453, 1.        , 0.03763058, 0.0345133 ]
[ 2.        , 0.0441524 , 1.        , 0.0345133 , 0.03933453]
[ 3.        , 0.0489669 , 1.        , 0.03933453, 0.0441524 ]
[ 4.        , 0.05689092, 1.        , 0.0441524 , 0.0489669 ]
[ 5.        , 0.05977237, 1.        , 0.0489669 , 0.05689092]
[ 6.        , 0.0547365 , 1.        , 0.05689092, 0.05977237]
[ 7.        , 0.05156805, 1.        , 0.05977237, 0.0547365 ]
[ 8.        , 0.04840135, 1.        , 0.0547365 , 0.05156805]
[ 9.        , 0.0452364 , 1.        , 0.05156805, 0.04840135]
[ 10.        ,  0.04710003,  1.        ,  0.04840135,  0.0452364 ]
[ 11.        ,  0.04102572,  1.        ,  0.0452364 ,  0.04710003]
[ 12.        ,  0.03309277,  1.        ,  0.04710003,  0.04102572]
[ 13.        ,  0.02664582,  1.        ,  0.04102572,  0.03309277]
[ 14.        ,  0.0202037 ,  1.        ,  0.03309277,  0.02664582]
[  1.50000000e+01,  

[  9.90000000e+01,  6.94057500e-03,  1.00000000e+00,  7.01034832e-03,
   6.97522258e-03]
[  1.00000000e+02,  6.90640556e-03,  1.00000000e+00,  6.97522258e-03,
   6.94057500e-03]
[  1.01000000e+02,  8.01582145e-03,  1.00000000e+00,  6.94057500e-03,
   6.90640556e-03]
[  1.02000000e+02,  7.39819761e-03,  1.00000000e+00,  6.90640556e-03,
   8.01582145e-03]
[  1.03000000e+02,  6.79417478e-03,  1.00000000e+00,  8.01582145e-03,
   7.39819761e-03]
[  1.04000000e+02,  6.19005515e-03,  1.00000000e+00,  7.39819761e-03,
   6.79417478e-03]
[  1.05000000e+02,  5.58583872e-03,  1.00000000e+00,  6.79417478e-03,
   6.19005515e-03]
[  1.06000000e+02,  5.59953778e-03,  1.00000000e+00,  6.19005515e-03,
   5.58583872e-03]
[  1.07000000e+02,  4.50475551e-03,  1.00000000e+00,  5.58583872e-03,
   5.59953778e-03]
[  1.08000000e+02,  3.39604041e-03,  1.00000000e+00,  5.59953778e-03,
   4.50475551e-03]
[  1.09000000e+02,  2.28747233e-03,  1.00000000e+00,  4.50475551e-03,
   3.39604041e-03]
[  1.10000000e+02,  1

[  1.92000000e+02,  1.18629736e-02,  1.00000000e+00,  8.98668019e-03,
   1.18922877e-02]
[  1.93000000e+02,  1.92867628e-02,  1.00000000e+00,  1.18922877e-02,
   1.18629736e-02]
[  1.94000000e+02,  2.67064852e-02,  1.00000000e+00,  1.18629736e-02,
   1.92867628e-02]
[  1.95000000e+02,  3.41221441e-02,  1.00000000e+00,  1.92867628e-02,
   2.67064852e-02]
[  1.96000000e+02,  4.15337429e-02,  1.00000000e+00,  2.67064852e-02,
   3.41221441e-02]
[  1.97000000e+02,  4.89705939e-02,  1.00000000e+00,  3.41221441e-02,
   4.15337429e-02]
[  1.98000000e+02,  4.60326522e-02,  1.00000000e+00,  4.15337429e-02,
   4.89705939e-02]
[  1.99000000e+02,  4.31802504e-02,  1.00000000e+00,  4.89705939e-02,
   4.60326522e-02]
[  2.00000000e+02,  4.03288500e-02,  1.00000000e+00,  4.60326522e-02,
   4.31802504e-02]
[  2.01000000e+02,  3.74784506e-02,  1.00000000e+00,  4.31802504e-02,
   4.03288500e-02]
[  2.02000000e+02,  3.75629688e-02,  1.00000000e+00,  4.03288500e-02,
   3.74784506e-02]
[  2.03000000e+02,  3

[  2.85000000e+02,  5.30878478e-03,  1.00000000e+00,  2.08539990e-02,
   1.30787808e-02]
[  2.86000000e+02,  6.70362143e-03,  1.00000000e+00,  1.30787808e-02,
   5.30878478e-03]
[  2.87000000e+02,  7.87371175e-03,  1.00000000e+00,  5.30878478e-03,
   6.70362143e-03]
[  2.88000000e+02,  7.64927358e-03,  1.00000000e+00,  6.70362143e-03,
   7.87371175e-03]
[  2.89000000e+02,  1.31737489e-02,  1.00000000e+00,  7.87371175e-03,
   7.64927358e-03]
[  2.90000000e+02,  1.86954471e-02,  1.00000000e+00,  7.64927358e-03,
   1.31737489e-02]
[  2.91000000e+02,  2.42143702e-02,  1.00000000e+00,  1.31737489e-02,
   1.86954471e-02]
[  2.92000000e+02,  2.99550434e-02,  1.00000000e+00,  1.86954471e-02,
   2.42143702e-02]
[  2.93000000e+02,  3.42971411e-02,  1.00000000e+00,  2.42143702e-02,
   2.99550434e-02]
[  2.94000000e+02,  3.86368206e-02,  1.00000000e+00,  2.99550434e-02,
   3.42971411e-02]
[  2.95000000e+02,  3.72416544e-02,  1.00000000e+00,  3.42971411e-02,
   3.86368206e-02]
[  2.96000000e+02,  3

[  3.78000000e+02,  1.76183165e-02,  1.00000000e+00,  2.93768600e-02,
   2.34963061e-02]
[  3.79000000e+02,  1.17804100e-02,  1.00000000e+00,  2.34963061e-02,
   1.76183165e-02]
[  3.80000000e+02,  5.94535190e-03,  1.00000000e+00,  1.76183165e-02,
   1.17804100e-02]
[  3.81000000e+02,  1.13140292e-04,  1.00000000e+00,  1.17804100e-02,
   5.94535190e-03]
[  3.82000000e+02,  1.50378444e-04,  1.00000000e+00,  5.94535190e-03,
   1.13140292e-04]
[  3.83000000e+02,  1.87331141e-04,  1.00000000e+00,  1.13140292e-04,
   1.50378444e-04]
[  3.84000000e+02,  1.87044248e-04,  1.00000000e+00,  1.50378444e-04,
   1.87331141e-04]
[  3.85000000e+02,  1.87791813e-04,  1.00000000e+00,  1.87331141e-04,
   1.87044248e-04]
[  3.86000000e+02,  1.88539386e-04,  1.00000000e+00,  1.87044248e-04,
   1.87791813e-04]
[  3.87000000e+02,  1.89286966e-04,  1.00000000e+00,  1.87791813e-04,
   1.88539386e-04]
[  3.88000000e+02,  1.90321443e-04,  1.00000000e+00,  1.88539386e-04,
   1.89286966e-04]
[  3.89000000e+02,  1

[  4.71000000e+02,  6.29337632e-06,  1.00000000e+00,  7.77539310e-05,
   4.20243291e-05]
[  4.72000000e+02,  6.29725401e-06,  1.00000000e+00,  4.20243291e-05,
   6.29337632e-06]
[  4.73000000e+02,  5.04480087e-06,  1.00000000e+00,  6.29337632e-06,
   6.29725401e-06]
[  4.74000000e+02,  3.78846113e-06,  1.00000000e+00,  6.29725401e-06,
   5.04480087e-06]
[  4.75000000e+02,  2.27232271e-03,  1.00000000e+00,  5.04480087e-06,
   3.78846113e-06]
[  4.76000000e+02,  4.54017319e-03,  1.00000000e+00,  3.78846113e-06,
   2.27232271e-03]
[  4.77000000e+02,  6.80734019e-03,  1.00000000e+00,  2.27232271e-03,
   4.54017319e-03]
[  4.78000000e+02,  9.07508040e-03,  1.00000000e+00,  4.54017319e-03,
   6.80734019e-03]
[  4.79000000e+02,  1.13421339e-02,  1.00000000e+00,  6.80734019e-03,
   9.07508040e-03]
[  4.80000000e+02,  1.13421300e-02,  1.00000000e+00,  9.07508040e-03,
   1.13421339e-02]
[  4.81000000e+02,  1.43475328e-02,  1.00000000e+00,  1.13421339e-02,
   1.13421300e-02]
[  4.82000000e+02,  1

[  5.64000000e+02,  4.92342007e-02,  1.00000000e+00,  3.93750613e-02,
   4.92342261e-02]
[  5.65000000e+02,  4.92786186e-02,  1.00000000e+00,  4.92342261e-02,
   4.92342007e-02]
[  5.66000000e+02,  4.93230383e-02,  1.00000000e+00,  4.92342007e-02,
   4.92786186e-02]
[  5.67000000e+02,  4.93674600e-02,  1.00000000e+00,  4.92786186e-02,
   4.93230383e-02]
[  5.68000000e+02,  4.94119089e-02,  1.00000000e+00,  4.93230383e-02,
   4.93674600e-02]
[  5.69000000e+02,  3.96218886e-02,  1.00000000e+00,  4.93674600e-02,
   4.94119089e-02]
[  5.70000000e+02,  2.97812465e-02,  1.00000000e+00,  4.94119089e-02,
   3.96218886e-02]
[  5.71000000e+02,  2.06849130e-02,  1.00000000e+00,  3.96218886e-02,
   2.97812465e-02]
[  5.72000000e+02,  1.15825268e-02,  1.00000000e+00,  2.97812465e-02,
   2.06849130e-02]
[  5.73000000e+02,  2.47408221e-03,  1.00000000e+00,  2.06849130e-02,
   1.15825268e-02]
[  5.74000000e+02,  3.22495802e-03,  1.00000000e+00,  1.15825268e-02,
   2.47408221e-03]
[  5.75000000e+02,  3

[  6.57000000e+02,  3.37171331e-03,  1.00000000e+00,  1.12686286e-03,
   2.24940722e-03]
[  6.58000000e+02,  4.49513279e-03,  1.00000000e+00,  2.24940722e-03,
   3.37171331e-03]
[  6.59000000e+02,  5.61830908e-03,  1.00000000e+00,  3.37171331e-03,
   4.49513279e-03]
[  6.60000000e+02,  6.74124733e-03,  1.00000000e+00,  4.49513279e-03,
   5.61830908e-03]
[  6.61000000e+02,  6.74124223e-03,  1.00000000e+00,  5.61830908e-03,
   6.74124733e-03]
[  6.62000000e+02,  6.74788102e-03,  1.00000000e+00,  6.74124733e-03,
   6.74124223e-03]
[  6.63000000e+02,  6.75451970e-03,  1.00000000e+00,  6.74124223e-03,
   6.74788102e-03]
[  6.64000000e+02,  6.76116336e-03,  1.00000000e+00,  6.74788102e-03,
   6.75451970e-03]
[  6.65000000e+02,  5.64367265e-03,  1.00000000e+00,  6.75451970e-03,
   6.76116336e-03]
[  6.66000000e+02,  4.51977672e-03,  1.00000000e+00,  6.76116336e-03,
   5.64367265e-03]
[  6.67000000e+02,  6.62224662e-03,  1.00000000e+00,  5.64367265e-03,
   4.51977672e-03]
[  6.68000000e+02,  8